In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [1]:
!ls

Untitled.ipynb	     green_tripdata_2019-10.csv.gz
docker-compose.yaml  taxi_zone_lookup.csv


In [3]:
pg_user="postgres"
pg_pass="postgres"
pg_host="localhost"
pg_port=5433
pg_db="ny_taxi"

In [4]:
engine = create_engine(f"postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}")

In [5]:
con = engine.connect()

#test connection
con

In [6]:
df_iter = pd.read_csv("green_tripdata_2019-10.csv.gz", iterator=True, chunksize=100000)
df = next(df_iter)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   lpep_pickup_datetime   100000 non-null  object 
 2   lpep_dropoff_datetime  100000 non-null  object 
 3   store_and_fwd_flag     100000 non-null  object 
 4   RatecodeID             100000 non-null  int64  
 5   PULocationID           100000 non-null  int64  
 6   DOLocationID           100000 non-null  int64  
 7   passenger_count        100000 non-null  int64  
 8   trip_distance          100000 non-null  float64
 9   fare_amount            100000 non-null  float64
 10  extra                  100000 non-null  float64
 11  mta_tax                100000 non-null  float64
 12  tip_amount             100000 non-null  float64
 13  tolls_amount           100000 non-null  float64
 14  ehail_fee              0 non-null    

In [8]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

In [9]:
df.head(0).to_sql(con = con, name = "green_tripdata_2019-10", if_exists="replace")

0

In [10]:
df.to_sql(con = con, name = "green_tripdata_2019-10", if_exists="append")

1000

In [ ]:
try:
    while True:
        # Fetch the next chunk of data
        df = next(df_iter)

        # Convert datetime columns
        df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
        df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

        # Append the chunk to the SQL table
        df.to_sql(con=con, name="green_tripdata_2019_10", if_exists="append", index=False)

except StopIteration:
    print("Data loading complete.")
except Exception as e:
    print(f"An error occurred: {e}")

# Close the database connection
finally:
    con.close()